In [ ]:
# =====================================================
# Napy Token Vault Architecture Analysis (Data Analyst Edition)
# =====================================================

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import Dict, List

# =============================================
# 1. Core Architecture Constants
# =============================================
class VaultConstants:
    MAX_BPS = 10_000  # 100%
    MAXIMUM_STRATEGIES = 20
    DEGRADATION_COEFFICIENT = 10**18
    SECS_PER_YEAR = 31_556_952  # 365.2425 days

    PERFORMANCE_FEE_BPS = 1_000  # 10%
    MANAGEMENT_FEE_BPS = 200     # 2%


print("Vault Core Constants Initialized.")


# =============================================
# 2. Data Structures
# =============================================
@dataclass
class StrategyParams:
    performanceFee: int
    activation: int
    debtRatio: int
    minDebtPerHarvest: int
    maxDebtPerHarvest: int
    lastReport: int
    totalDebt: int
    totalGain: int
    totalLoss: int


@dataclass
class VaultState:
    totalAssets: int
    totalIdle: int
    totalDebt: int
    totalSupply: int
    lockedProfit: int
    debtRatio: int
    depositLimit: int


# =============================================
# 3. Core Calculations
# =============================================
class VaultCalculations:

    @staticmethod
    def calculate_locked_profit(locked_profit, last_report, current_time, degradation_rate):
        elapsed_time = current_time - last_report
        degradation = elapsed_time * degradation_rate
        if degradation < VaultConstants.DEGRADATION_COEFFICIENT:
            return locked_profit - (degradation * locked_profit) // VaultConstants.DEGRADATION_COEFFICIENT
        else:
            return 0

    @staticmethod
    def shares_for_amount(amount, free_funds, total_supply):
        return (amount * total_supply) // free_funds if free_funds > 0 else amount

    @staticmethod
    def share_value(shares, free_funds, total_supply):
        if total_supply == 0:
            return shares
        return (shares * free_funds) // total_supply

    @staticmethod
    def calculate_credit_available(
        strategy_debt_ratio, strategy_total_debt, vault_total_assets,
        vault_debt_ratio, vault_total_debt, total_idle, min_debt, max_debt
    ):
        strategy_debt_limit = (strategy_debt_ratio * vault_total_assets) // VaultConstants.MAX_BPS
        vault_debt_limit = (vault_debt_ratio * vault_total_assets) // VaultConstants.MAX_BPS

        if strategy_debt_limit <= strategy_total_debt or vault_debt_limit <= vault_total_debt:
            return 0

        available = min(
            strategy_debt_limit - strategy_total_debt,
            vault_debt_limit - vault_total_debt,
            total_idle
        )

        return min(max(available, 0), max_debt)


# =============================================
# 4. Fee Calculation
# =============================================
class FeeCalculator:

    @staticmethod
    def calculate_management_fee(strategy_debt, delegated_assets, duration, management_fee_bps):
        effective_debt = max(strategy_debt - delegated_assets, 0)
        return (effective_debt * duration * management_fee_bps) // (VaultConstants.MAX_BPS * VaultConstants.SECS_PER_YEAR)

    @staticmethod
    def calculate_performance_fee(gain, performance_fee_bps):
        return (gain * performance_fee_bps) // VaultConstants.MAX_BPS

    @staticmethod
    def assess_fees(gain, strategy_debt, delegated_assets, duration,
                    strategy_performance_fee, vault_performance_fee, vault_management_fee):
        if gain <= 0:
            return {'management_fee': 0, 'strategist_fee': 0, 'performance_fee': 0, 'total_fee': 0}

        management_fee = FeeCalculator.calculate_management_fee(strategy_debt, delegated_assets, duration, vault_management_fee)
        strategist_fee = FeeCalculator.calculate_performance_fee(gain, strategy_performance_fee)
        performance_fee = FeeCalculator.calculate_performance_fee(gain, vault_performance_fee)
        total_fee = min(management_fee + strategist_fee + performance_fee, gain)

        return {
            'management_fee': management_fee,
            'strategist_fee': strategist_fee,
            'performance_fee': performance_fee,
            'total_fee': total_fee
        }


# =============================================
# 5. Data Analyst: Generate Multiple Scenarios
# =============================================

def simulate_vault_scenarios(num_samples=50):
    np.random.seed(42)
    data = []
    for _ in range(num_samples):
        gain = np.random.randint(-200_000, 2_000_000)
        strategy_debt = np.random.randint(1_000_000, 10_000_000)
        delegated_assets = np.random.randint(100_000, 2_000_000)
        duration = np.random.randint(1, 2_592_000)  # up to 30 days
        result = FeeCalculator.assess_fees(
            gain=gain,
            strategy_debt=strategy_debt,
            delegated_assets=delegated_assets,
            duration=duration,
            strategy_performance_fee=1000,
            vault_performance_fee=1000,
            vault_management_fee=200
        )
        result.update({
            'gain': gain,
            'strategy_debt': strategy_debt,
            'delegated_assets': delegated_assets,
            'duration': duration
        })
        data.append(result)
    df = pd.DataFrame(data)
    df['net_gain_after_fee'] = df['gain'] - df['total_fee']
    return df


df = simulate_vault_scenarios()
print("\n=== Vault Fee Simulation Data ===")
print(df.head())


# =============================================
# 6. Analytical Summary
# =============================================
print("\n=== Statistical Summary ===")
print(df.describe())

corr = df[['gain', 'total_fee', 'management_fee', 'performance_fee', 'net_gain_after_fee']].corr()
print("\n=== Correlation Matrix ===")
print(corr)


# =============================================
# 7. Visualization (Analyst View)
# =============================================
def visualize_analysis(df):
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))

    axs[0, 0].hist(df['gain'], bins=20)
    axs[0, 0].set_title("Gain Distribution")

    axs[0, 1].scatter(df['gain'], df['total_fee'], alpha=0.7)
    axs[0, 1].set_title("Gain vs Total Fee")

    axs[1, 0].bar(['Mgmt', 'Perf', 'Strat'], [
        df['management_fee'].mean(),
        df['performance_fee'].mean(),
        df['strategist_fee'].mean()
    ])
    axs[1, 0].set_title("Average Fee Components")

    axs[1, 1].plot(df['net_gain_after_fee'].cumsum())
    axs[1, 1].set_title("Cumulative Net Gain Over Time")

    plt.tight_layout()
    plt.show()


visualize_analysis(df)


# =============================================
# 8. Test Cases (Validation Layer)
# =============================================
def run_tests():
    print("\n=== Running Test Cases ===")
    # Case 1: No gain -> all fees should be zero
    res = FeeCalculator.assess_fees(0, 1_000_000, 0, 3600, 1000, 1000, 200)
    assert all(v == 0 for v in res.values()), "Failed: Zero gain should result in zero fees"

    # Case 2: Small gain should have small fee
    res = FeeCalculator.assess_fees(100_000, 2_000_000, 500_000, 604_800, 1000, 1000, 200)
    assert res['total_fee'] < 100_000, "Failed: Fees exceed gain"

    # Case 3: Negative gain (loss)
    res = FeeCalculator.assess_fees(-100_000, 2_000_000, 500_000, 604_800, 1000, 1000, 200)
    assert res['total_fee'] == 0, "Failed: Loss should have zero fee"

    # Case 4: High gain scenario
    res = FeeCalculator.assess_fees(2_000_000, 10_000_000, 0, 2_592_000, 1500, 1000, 200)
    assert res['total_fee'] < 2_000_000, "Failed: Total fee exceeds gain"

    print("✅ All test cases passed.")


run_tests()

# =============================================
# 10. Long-Term Compounding Simulation (20 Years)
# =============================================
def simulate_compounding(
    initial_investment=1_000_000,
    avg_gain_rate=0.10,  # 10% annual yield before fees
    performance_fee_bps=VaultConstants.PERFORMANCE_FEE_BPS,
    management_fee_bps=VaultConstants.MANAGEMENT_FEE_BPS,
    strategy_performance_fee=VaultConstants.PERFORMANCE_FEE_BPS,
    duration_years=20,
    compounding_periods_per_year=12
):
    """
    Simulate the compounding effect of vault returns over 20 years with monthly compounding.
    """
    balance_gross = [initial_investment]
    balance_net = [initial_investment]
    fee_history = []
    period_gain_rate = avg_gain_rate / compounding_periods_per_year

    for i in range(duration_years * compounding_periods_per_year):
        # Period gain before fees
        gross_gain = balance_net[-1] * period_gain_rate
        # Fees applied based on current balance
        fee_data = FeeCalculator.assess_fees(
            gain=int(gross_gain),
            strategy_debt=int(balance_net[-1]),
            delegated_assets=0,
            duration=int(VaultConstants.SECS_PER_YEAR / compounding_periods_per_year),
            strategy_performance_fee=strategy_performance_fee,
            vault_performance_fee=performance_fee_bps,
            vault_management_fee=management_fee_bps
        )
        net_gain = gross_gain - fee_data['total_fee']
        balance_gross.append(balance_gross[-1] + gross_gain)
        balance_net.append(balance_net[-1] + net_gain)
        fee_history.append(fee_data['total_fee'])

    df_compound = pd.DataFrame({
        'Year': np.arange(0, duration_years + 1/compounding_periods_per_year, 1/compounding_periods_per_year)[:len(balance_net)],
        'Balance_Gross': balance_gross,
        'Balance_Net': balance_net,
        'Fees': [0] + fee_history
    })

    print("\n=== Compounding Summary (20 Years) ===")
    print(f"Initial Investment: {initial_investment:,.2f}")
    print(f"Final Gross Balance: {df_compound['Balance_Gross'].iloc[-1]:,.2f}")
    print(f"Final Net Balance (after fees): {df_compound['Balance_Net'].iloc[-1]:,.2f}")
    print(f"Total Fees Paid: {sum(fee_history):,.2f}")
    print(f"Effective Annualized Return (Net): {((df_compound['Balance_Net'].iloc[-1]/initial_investment)**(1/duration_years)-1)*100:.2f}%")

    # Visualization
    plt.figure(figsize=(10,6))
    plt.plot(df_compound['Year'], df_compound['Balance_Gross'], label="Gross Growth (Before Fees)", linestyle='--')
    plt.plot(df_compound['Year'], df_compound['Balance_Net'], label="Net Growth (After Fees)", linewidth=2)
    plt.title("Vault 20-Year Compounding Growth (Gross vs Net)")
    plt.xlabel("Years")
    plt.ylabel("Vault Value (Token Units)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return df_compound


# Run compounding simulation
compound_df = simulate_compounding()

# Optional: Summary Table
summary = compound_df.iloc[::12, :].copy()  # sample yearly
summary['Year'] = summary['Year'].round(1)
print("\n=== Yearly Growth Snapshot ===")
print(summary[['Year', 'Balance_Gross', 'Balance_Net']].head(20))



# =============================================
# 9. Vault Measurement Report (Human-Readable Summary)
# =============================================
measurement_report = """
================= Vault Measurement Report =================

Vault Core Constants:
MAX_BPS: 10000
MAXIMUM_STRATEGIES: 20
Performance Fee: 10.0%
Management Fee: 2.0%

Example Strategy Configuration:
Debt Ratio: 30.0%
Performance Fee: 10.0%
Current Debt: 2000000
Total Gain: 500000

Locked profit after 1 hour: 417200
Free funds: 9582800
Shares for 1,000,000 deposit: 887005

Fee Breakdown for 1M Gain:
management_fee: 1,533
strategist_fee: 100,000
performance_fee: 100,000
total_fee: 201,533

=== Basic Vault Deployment ===
token: 0xERC20_TOKEN_ADDRESS
governance: 0xGOVERNANCE_ADDRESS
rewards: 0xREWARDS_ADDRESS
guardian: 0xGUARDIAN_ADDRESS
management: 0xMANAGEMENT_ADDRESS

=== Strategy Integration ===
Total Debt Ratio: 90.0%
  - Compound: 40.0%
  - Aave: 30.0%
  - Curve: 20.0%

=== User Workflow ===
Deposit 1000000 → 947368 shares
Withdraw 500000 shares → 527777

=== Withdrawal Risk Analysis ===
Loss: 0.3%

=== Concentration Risk Analysis ===
Herfindahl Index: 3075.00
============================================================
"""

print(measurement_report)
